In [25]:
!pip install pyinterval

In [26]:
from interval import interval
import pandas as pd

In [27]:
def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_middle_point(interv):
    return (interv[0].sup + interv[0].inf) / 2

def to_string(interv):
    return f"[{interv[0].inf:9.7f}, {interv[0].sup:9.7f}]"

In [37]:
def calculate_volterra2(f, expected, k, kdf, y0, a, b, c, d, n):
    h = (b - a) / n
    x = []
    y = []
    exp = []
    x.append(a)
    y.append(y0)
    exp.append(expected(a))
    for i in range(1, n+1):
        x.append(a + i*h)
        sum = 0
        for j in range(i):
            sum += k(x[i], x[j], y[j])
        y.append(f(x[i]) + h*sum + (h*(x[i]-a)/2)*kdf(x[i], interval[a, x[i]], interval[c, d]))
        exp.append(expected(x[i]))
    return x, y, exp

In [38]:
a = 0
b = 1/4
c = 0
d = 2
n = 10
y0 = interval[0,2]
func = lambda x: 1
k = lambda x,s,y : y**2
kdf = lambda x,s,y : 2*y
expected = lambda x: 1/(1-x)

# Riemann sum

In [40]:
tbl2 = pd.DataFrame(columns=['xi', 'yi', 'm(yi)', 'expected', 'e'])
x, y, exp = calculate_volterra2(func, expected, k, kdf, y0, a, b, c, d, n)
for i in range(n+1):
    mid_point=calculate_middle_point(y[i])
    tbl2.loc[len(tbl2)] = [f"{x[i]:9.4f}", to_string(y[i]), f"{mid_point:9.7f}", f"{exp[i]:9.7f}", abs(mid_point-exp[i])]
tbl2

,xi,yi,m(yi),expected,e
0,0.0000,"[0.0000000, 2.0000000]",1.0000000,1.0000000,0.000000
1,0.0250,"[1.0000000, 1.1012500]",1.0506250,1.0256410,0.024984
2,0.0500,"[1.0250000, 1.1328188]",1.0789094,1.0526316,0.026278
3,0.0750,"[1.0512656, 1.1661507]",1.1087082,1.0810811,0.027627
4,0.1000,"[1.0788946, 1.2013984]",1.1401465,1.1111111,0.029035
5,0.1250,"[1.1079949, 1.2387324]",1.1733637,1.1428571,0.030507
6,0.1500,"[1.1386863, 1.2783438]",1.2085151,1.1764706,0.032044
7,0.1750,"[1.1711014, 1.3204479]",1.2457747,1.2121212,0.033653
8,0.2000,"[1.2053884, 1.3652875]",1.2853379,1.2500000,0.035338
9,0.2250,"[1.2417124, 1.4131377]",1.3274251,1.2903226,0.037102
